### Imports

In [97]:
import pandas as pd

import requests as rq
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Keys

In [2]:
# Get demo API key
def get_demo_key():
    f = open("/home/vikas/Documents/CG_demo_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [3]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

### API status

In [7]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [76]:
def get_response(endpoint, headers, params, URL):
    
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [57]:
use_demo = {
           "accept": "application/json",
           "x-cg-demo-api-key" : get_demo_key() 
}

use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [10]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

#### List of coins

In [12]:
def get_list_of_coins():

    response = get_response("/coins/list", use_demo, "", PUB_URL)

    return pd.DataFrame(response) 

In [15]:
df_coins = get_list_of_coins()

df_coins[df_coins["symbol"] == "eth"]

,id,symbol,name
2045,bifrost-bridged-eth-bifrost,eth,Bifrost Bridged ETH (Bifrost)
2803,bridged-binance-peg-ethereum-opbnb,eth,Bridged Binance-Peg Ethereum (opBNB)
2863,bridged-wrapped-ether-eclipse,eth,Bridged Wrapped Ether (Eclipse)
2872,bridged-wrapped-ether-starkgate,eth,Bridged Ether (StarkGate)
5839,ethereum,eth,Ethereum
5854,ethereum-wormhole,eth,Ethereum (Wormhole)
8071,immutable-zkevm-bridged-eth,eth,Immutable zkEVM Bridged ETH
9106,laika-bridged-eth-laika,eth,Laika Bridged ETH (Laika)
11522,nova-eth,eth,Nova Merged ETH (zkLink)
11994,osmosis-alleth,eth,Osmosis allETH


### OHLC

In [80]:
def get_ohlc_data(coin_id,
                  target_curr,
                  days,
                  precision,
                  interval = "hourly"):
    
    ohlc_params = {
        "vs_currency": target_curr,
        "days": days,
        "precision": precision,
        #"interval": interval
    }

    ohlc_list_response = get_response(f"/coins/{coin_id}/ohlc",
                                      use_pro,
                                      ohlc_params,
                                      PRO_URL)

    # Convert to DataFrame and assign column names
    df_ohlc = pd.DataFrame(ohlc_list_response,
                           columns=["timestamp", "open", "high", "low", "close"])

    # Convert timestamp from milliseconds to datetime
    df_ohlc["timestamp"] = pd.to_datetime(df_ohlc["timestamp"], unit='ms')

    return df_ohlc

In [84]:
#get_ohlc_data("bitcoin", "usd", "max", "2")

#### Plot candlestick

In [45]:
def plot_candlestick(coin_id,
                     target_curr,
                     days,
                     precision,
                     ):

    df_ohlc = get_ohlc_data(coin_id, 
                            target_curr,
                            days,
                            precision)

    target_curr = target_curr.upper()
    
    fig = go.Figure(
        data=[
            go.Candlestick(
                x = df_ohlc["timestamp"],
                open = df_ohlc["open"],
                high = df_ohlc["high"],
                low  = df_ohlc["low"],
                close = df_ohlc["close"],
                name = f"Price [{target_curr}]",
            )
        ]
    )

    fig.update_layout(
        title={
        'text': f'Candlestick chart for {coin_id} over {days} days',
        'x': 0.5,
        'xanchor': 'center'
        },
        xaxis_title = "Date",
        yaxis_title = f"Price [{target_curr}]",
        xaxis_rangeslider_visible = False,
        # Optional: use "plotly", "ggplot2", "seaborn", etc.
        template = "plotly",
        height = 600
    )

    fig.show()

    return None

In [46]:
plot_candlestick("bitcoin", "eur", "365", "2")

#### SMA

In [51]:
def add_sma(df_ohlc, window_short, window_long):

    df_ohlc['sma_short'] = df_ohlc['close'].rolling(window = window_short).mean()
    df_ohlc['sma_long'] = df_ohlc['close'].rolling(window = window_long).mean()

    return df_ohlc

#### Plot candlestick + SMA

In [90]:
def plot_candlestick_sma(coin_id,
                         target_curr,
                         days,
                         precision,
                         window_short,
                         window_long
                         ):

    df_ohlc = get_ohlc_data(coin_id, 
                            target_curr,
                            days,
                            precision)
    
    df_ohlc = add_sma(df_ohlc, window_short, window_long)

    target_curr = target_curr.upper()   

    fig = go.Figure()
    
    # Candlestick trace
    fig.add_trace(go.Candlestick(
        x = df_ohlc['timestamp'],
        open = df_ohlc['open'],
        high = df_ohlc['high'],
        low = df_ohlc['low'],
        close = df_ohlc['close'],
        name = 'OHLC'
    ))

    # SMA short window
    fig.add_trace(go.Scatter(
        x = df_ohlc['timestamp'],
        y = df_ohlc['sma_short'],
        mode = 'lines',
        line = dict(color = 'blue', width = 1),
        name = f"SMA {window_short}"
    ))

    # SMA 200 trace
    fig.add_trace(go.Scatter(
        x = df_ohlc['timestamp'],
        y = df_ohlc['sma_long'],
        mode = 'lines',
        line = dict(color = 'orange', width = 1),
        name = f"SMA {window_long}"
    ))

    # Update layout
    fig.update_layout(
        title=dict(
            text = f"Candlestick chart with SMA {window_short} & SMA {window_long}",
            x = 0.5,  # Center title
            xanchor = 'center'
        ),
        xaxis_title = 'Date',
        yaxis_title = f'Price [{target_curr}]',
        xaxis_rangeslider_visible = False,
        template = "plotly",
        height = 600
    )

    fig.show()

    return None

In [91]:
plot_candlestick_sma("bitcoin", "eur", "max", "2", 50, 200)

#### RSI

In [92]:
def add_rsi(df,
            period = 14,
            column = 'close'):
    
    delta = df[column].diff()

    # Positive gains (up) and losses (down)
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    # Use exponential moving average (EMA) for smoothing
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    df['rsi'] = rsi
    return df


In [94]:
#add_rsi(get_ohlc_data("bitcoin", "usd", "365", "2"))

#### Plot candlestick + RSI

In [117]:
def plot_candlestick_rsi(coin_id,
                         target_curr,
                         days,
                         precision,
                         ):

    df_ohlc = get_ohlc_data(coin_id, 
                            target_curr,
                            days,
                            precision)
    
    df_ohlc = add_rsi(df_ohlc)

    target_curr = target_curr.upper()

    # Create subplot with 2 rows: candlestick and RSI
    fig = make_subplots(
        rows = 2,
        cols = 1,
        shared_xaxes = True,
        shared_yaxes = False,        
        row_heights = [0.5, 0.5],
        vertical_spacing = 0.1,
        subplot_titles = ("Candlestick chart",
                          "Relative Strength Index (RSI)")
    )
    
    fig.add_trace(
        go.Candlestick(
            x=df_ohlc['timestamp'],
            open=df_ohlc['open'],
            high=df_ohlc['high'],
            low=df_ohlc['low'],
            close=df_ohlc['close'],
            name='OHLC'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df_ohlc['timestamp'],
            y=df_ohlc['rsi'],
            mode='lines',
            line=dict(color='green'),
            name='RSI'
        ),
        row=2, col=1
    )

    # Add overbought/oversold RSI lines (70/30)
    fig.add_hline(y = 70, line = dict(color = "red", dash = "dash"), row = 2, col = 1)
    fig.add_hline(y = 30, line = dict(color = "blue", dash = "dash"), row = 2, col = 1)

    # Layout settings
    fig.update_layout(
        title={
            'text': f'Candlestick and RSI chart for {coin_id} over {days} days',
            'x': 0.5,
            'xanchor': 'center'
        },
        #xaxis_title = 'Date',
        yaxis_title = f"Price [{target_curr}]",
        yaxis2_title = 'RSI',
        height = 1000,
        template = "plotly"
    )

    # # Decouple y-axis from top plot
    # fig.update_yaxes(matches=None, row=2, col=1)  
    # fig.update_xaxes(title_text="Date", row=2, col=1)
    # fig.update_yaxes(title_text="RSI", row=2, col=1)

    # # Optional: Set y-axis range for RSI to standard bounds
    # fig.update_yaxes(range=[0, 100], row=2, col=1)

    fig.show()

    return None

In [118]:
plot_candlestick_rsi("bitcoin", "eur", "365", "2")